# Use Obspy to read and write station inventory files

- Read station XML files with extra metadata in Json format 
- modify existing metadata values
- add new metadata items such as Equipments

## Fei Zhang

* CreationDate: 2020-07-14
* LastUpdated:  2020-09-10

# Ref:

* https://docs.obspy.org/tutorial/code_snippets/stationxml_file_from_scratch.html
* https://docs.obspy.org/packages/autogen/obspy.core.inventory.util.Equipment.__init__.html


In [1]:
!pwd

/Softlab/Githubz/hiperseis/notebooks


In [2]:
import os
import sys
from obspy import read_inventory
from obspy.core import UTCDateTime

dirpath = "/g/data/ha3/Passive/SHARED_DATA/Inventory/Station_Extra_Metadata/NewInventoryXML"  # if in NCI
dirpath="/Datasets/Station_Extra_Metadata/NewInventoryXML"  # the dir where input_station xml file located
xmlfile_name= "OA.CE28_station_metadata_JSON.xml"  

input_station_xml = os.path.join(dirpath, xmlfile_name)


# VS extracted from ASDF
#input_station_xml ="OA.CF28_station_inv_modified_json_extracted_from_ASDF.xml"

our_inv = read_inventory(input_station_xml,format='STATIONXML')


stn_meta = our_inv.networks[0].stations


In [3]:
print(type(stn_meta[0].extra))
print(stn_meta[0].extra)

<class 'obspy.core.util.attribdict.AttribDict'>
AttribDict({'GAMetadata': AttribDict({'namespace': 'https://github.com/GeoscienceAustralia/hiperseis', 'value': '{\n  "network": "OA",\n  "station": "CE28",\n  "GPS_CORRECTION": [\n    {\n      "date": "2017-12-01",\n      "clock_correction": 0.269773702\n    },\n    {\n      "date": "2017-12-02",\n      "clock_correction": 0.2650179231\n    },\n    {\n      "date": "2017-12-03",\n      "clock_correction": 0.2602621441\n    },\n    {\n      "date": "2017-12-04",\n      "clock_correction": 0.2555063652\n    },\n    {\n      "date": "2017-12-05",\n      "clock_correction": 0.2507505863\n    },\n    {\n      "date": "2017-12-06",\n      "clock_correction": 0.2459948074\n    },\n    {\n      "date": "2017-12-07",\n      "clock_correction": 0.2412390284\n    },\n    {\n      "date": "2017-12-08",\n      "clock_correction": 0.2364832495\n    },\n    {\n      "date": "2017-12-09",\n      "clock_correction": 0.2317274706\n    },\n    {\n      "da

In [4]:
# print(len(stn_meta))

print("type", type(stn_meta[0].start_date))
print(stn_meta[0].start_date)
print(stn_meta[0].end_date)

print(stn_meta[0].equipments)
print(stn_meta[0].creation_date, stn_meta[0].termination_date)
#print(stn_meta[0].extra) 

type <class 'obspy.core.utcdatetime.UTCDateTime'>
2017-11-06T02:31:42.000000Z
2018-06-09T05:29:53.000000Z
[]
2017-11-06T02:31:42.000000Z 2018-06-09T05:29:53.000000Z


In [5]:
# extra_meta = our_inv.networks[0].stations[0].extra.gpsclockcorrection.value

#extra_meta = stn_meta[2].extra.metadata_ga.value
extra_meta = stn_meta[0].extra.GAMetadata.value

print(type(extra_meta))

<class 'str'>


In [6]:
print(extra_meta)

{
  "network": "OA",
  "station": "CE28",
  "GPS_CORRECTION": [
    {
      "date": "2017-12-01",
      "clock_correction": 0.269773702
    },
    {
      "date": "2017-12-02",
      "clock_correction": 0.2650179231
    },
    {
      "date": "2017-12-03",
      "clock_correction": 0.2602621441
    },
    {
      "date": "2017-12-04",
      "clock_correction": 0.2555063652
    },
    {
      "date": "2017-12-05",
      "clock_correction": 0.2507505863
    },
    {
      "date": "2017-12-06",
      "clock_correction": 0.2459948074
    },
    {
      "date": "2017-12-07",
      "clock_correction": 0.2412390284
    },
    {
      "date": "2017-12-08",
      "clock_correction": 0.2364832495
    },
    {
      "date": "2017-12-09",
      "clock_correction": 0.2317274706
    },
    {
      "date": "2017-12-10",
      "clock_correction": 0.2269716916
    },
    {
      "date": "2017-12-11",
      "clock_correction": 0.2222159127
    },
    {
      "date": "2017-12-12",
      "clock_correction

In [7]:
import json

mdata = json.loads(extra_meta)

# print (mdata)
print(mdata.keys())
# print(type(mdata['gps_clock_corrections']))

print(type(mdata['GPS_CORRECTION']))

for corr in mdata['GPS_CORRECTION']:
    print (corr["date"], corr["clock_correction"], "seconds" )

dict_keys(['network', 'station', 'GPS_CORRECTION', 'ORIENT_CORRECTION'])
<class 'list'>
2017-12-01 0.269773702 seconds
2017-12-02 0.2650179231 seconds
2017-12-03 0.2602621441 seconds
2017-12-04 0.2555063652 seconds
2017-12-05 0.2507505863 seconds
2017-12-06 0.2459948074 seconds
2017-12-07 0.2412390284 seconds
2017-12-08 0.2364832495 seconds
2017-12-09 0.2317274706 seconds
2017-12-10 0.2269716916 seconds
2017-12-11 0.2222159127 seconds
2017-12-12 0.2174601338 seconds
2017-12-13 0.2127043548 seconds
2017-12-14 0.2079485759 seconds
2017-12-15 0.203192797 seconds
2017-12-16 0.198437018 seconds
2017-12-17 0.1936812391 seconds
2017-12-18 0.1889254602 seconds
2017-12-19 0.1841696813 seconds
2017-12-20 0.1794139023 seconds
2017-12-21 0.1746581234 seconds
2017-12-22 0.1699023445 seconds
2017-12-23 0.1651465655 seconds
2017-12-24 0.1603907866 seconds
2017-12-25 0.1556350077 seconds
2017-12-26 0.1508792287 seconds
2017-12-27 0.1461234498 seconds
2017-12-28 0.1413676709 seconds
2017-12-29 0.136611

In [8]:

print (mdata["ORIENT_CORRECTION"])

{}


In [9]:
# https://pbpython.com/pandas-list-dict.html

import pandas as pd

df_clock_correction = pd.DataFrame(mdata['GPS_CORRECTION'])
df_clock_correction.head()

,date,clock_correction
0,2017-12-01,0.269774
1,2017-12-02,0.265018
2,2017-12-03,0.260262
3,2017-12-04,0.255506
4,2017-12-05,0.250751


In [10]:
print (df_clock_correction.tail())

           date  clock_correction
151  2018-06-04        -10.547384
152  2018-06-05        -10.879123
153  2018-06-06        -11.214748
154  2018-06-07        -11.554780
155  2018-06-08        -11.899737


In [11]:
# get correction for a certain date YYYY-MM-DD
print(df_clock_correction.loc[df_clock_correction["date"] =="2018-06-06"])

           date  clock_correction
153  2018-06-06        -11.214748


In [12]:
type(df_clock_correction.query('date == "2018-06-06" '))

pandas.core.frame.DataFrame

In [13]:
df_clock_correction["date"] =="2018-06-06"

0      False
1      False
2      False
3      False
4      False
       ...  
151    False
152    False
153     True
154    False
155    False
Name: date, Length: 156, dtype: bool

In [14]:
df_clock_correction.query('date == "2018-06-06"').clock_correction

153   -11.214748
Name: clock_correction, dtype: float64

In [15]:
df_clock_correction.describe()


,clock_correction
count,156.000000
mean,-0.877534
std,5.408053
min,-13.673948
25%,-2.766375
50%,0.024851
75%,0.199626
max,15.991937


In [16]:
df_clock_correction.dtypes

date                 object
clock_correction    float64
dtype: object

In [17]:
# What if there are more than one Station Node, such as AU.HTT in testdata/network_AU_0.xml, and OA.CF28 
stations =our_inv.networks[0].stations
print (len(stations))

3


In [18]:
# wrtite out the inventory

#GA_NameSpace = "https://github.com/GeoscienceAustralia/hiperseis"
# how to get the extra's key 'GAMetadata'  
extram = list(stn_meta[0].extra)[0] 
print(extram)

GA_NameSpace = stn_meta[0].extra['GAMetadata']['namespace']

our_inv.write("our_inv.xml",format="stationxml",  nsmap={'GeoscienceAustralia': GA_NameSpace})

GAMetadata


In [19]:
!ls -lt *.xml 

-rw-r--r-- 1 fzhang fzhang 23337 Sep 18 20:12 our_inv.xml
-rw-r--r-- 1 fzhang fzhang 20157 Sep 18 10:54 station.xml
-rw-r--r-- 1 fzhang fzhang 23679 Sep 18 10:52 our_inv2.xml
-rwxr-xr-x 1 fzhang fzhang  7924 Jul 17 07:59 OA.CF28_station_inv_modified_json.xml
-rw-r--r-- 1 fzhang fzhang  6113 Jun 15 14:26 test_OA_CF28_inventory.xml
-rw-r--r-- 1 fzhang fzhang  6053 Jun 15 14:26 test_OA_CE28_inventory.xml
-rw-r--r-- 1 fzhang fzhang  6113 Jun 15 14:25 test_OA_BY22_inventory.xml
-rw-r--r-- 1 fzhang fzhang  4200 Jun 15 14:21 test_OAFZ_CE22_inventory.xml
-rw-r--r-- 1 fzhang fzhang  2309 Jun 15 14:20 test_OA_CE22_inventory.xml
-rw-r--r-- 1 fzhang fzhang 11879 Apr 10 09:28 modified_inventory_select.xml


In [20]:
my_cmd = "diff our_inv.xml " + input_station_xml
print(my_cmd)

diff our_inv.xml /Datasets/Station_Extra_Metadata/NewInventoryXML/OA.CE28_station_metadata_JSON.xml


In [21]:
os.system(my_cmd)  # =0 if success

0

In [22]:
# dir(our_inv)

type(our_inv)

obspy.core.inventory.inventory.Inventory

In [23]:
# try to analyze the extra metadata tags
print( type(stn_meta[0].extra))
print(dir(stn_meta[0].extra))
stn_meta[0].extra['GAMetadata']['namespace']

<class 'obspy.core.util.attribdict.AttribDict'>
['GAMetadata', '_MutableMapping__marker', '__abstractmethods__', '__class__', '__contains__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_cast_type', '_pretty_str', '_types', 'clear', 'copy', 'defaults', 'do_not_warn_on', 'get', 'items', 'keys', 'pop', 'popitem', 'readonly', 'setdefault', 'update', 'values', 'warn_on_non_default_key']


'https://github.com/GeoscienceAustralia/hiperseis'

In [24]:
list(stn_meta[0].extra)[0]

'GAMetadata'

# Modify the station XML file to include Equipments

In [25]:
# See also: https://docs.obspy.org/tutorial/code_snippets/stationxml_file_from_scratch.html

# Existing metadata values can be changed. 
import obspy
# stn_meta[0].start_date ="1990-01-01"
# stn_meta[0].end_date ="2029-07-01"

# print(stn_meta[0].start_date)
# print(stn_meta[0].end_date)

In [26]:
# New metadata (eg, Equipment) can be added 
# See https://gajira.atlassian.net/browse/PV-311

my_sensor=obspy.core.inventory.util.Equipment(type="Sensor", description="Nanometrics Trillium Compact 120s",serial_number="004940")
my_digitizer = obspy.core.inventory.util.Equipment(type="Digitizer", description="Guralp Minimus",serial_number="MIN-A456")

stn_meta[0].equipments = [my_sensor, my_digitizer]
print(stn_meta[0].equipments)

[<obspy.core.inventory.util.Equipment object at 0x7ff55f0df1d0>, <obspy.core.inventory.util.Equipment object at 0x7ff55f0df190>]


In [27]:
print(stn_meta[0].extra)


# Re-write out the inventory

our_inv.write("our_inv2.xml",format="stationxml", nsmap={'GeoscienceAustralia': GA_NameSpace})

AttribDict({'GAMetadata': AttribDict({'namespace': 'https://github.com/GeoscienceAustralia/hiperseis', 'value': '{\n  "network": "OA",\n  "station": "CE28",\n  "GPS_CORRECTION": [\n    {\n      "date": "2017-12-01",\n      "clock_correction": 0.269773702\n    },\n    {\n      "date": "2017-12-02",\n      "clock_correction": 0.2650179231\n    },\n    {\n      "date": "2017-12-03",\n      "clock_correction": 0.2602621441\n    },\n    {\n      "date": "2017-12-04",\n      "clock_correction": 0.2555063652\n    },\n    {\n      "date": "2017-12-05",\n      "clock_correction": 0.2507505863\n    },\n    {\n      "date": "2017-12-06",\n      "clock_correction": 0.2459948074\n    },\n    {\n      "date": "2017-12-07",\n      "clock_correction": 0.2412390284\n    },\n    {\n      "date": "2017-12-08",\n      "clock_correction": 0.2364832495\n    },\n    {\n      "date": "2017-12-09",\n      "clock_correction": 0.2317274706\n    },\n    {\n      "date": "2017-12-10",\n      "clock_correction": 0.

In [28]:
# compare of the two inventory files

!diff our_inv.xml our_inv2.xml

646a647,656
>       <Equipment>
>         <Type>Sensor</Type>
>         <Description>Nanometrics Trillium Compact 120s</Description>
>         <SerialNumber>004940</SerialNumber>
>       </Equipment>
>       <Equipment>
>         <Type>Digitizer</Type>
>         <Description>Guralp Minimus</Description>
>         <SerialNumber>MIN-A456</SerialNumber>
>       </Equipment>


In [29]:
#STOP!!

# Dealing with multiple station-nodes in a network.station inventory file

See Jira: https://gajira.atlassian.net/browse/PV-130

 What if there are more than one Station Node, such as AU.HTT in testdata/network_AU_0.xml. 
 And input_station_xml ="OA.CF28_station_inv_modified_json.xml"
 
The multple station code represent the same station with different channel configuraiton over different time periods.
 

In [39]:

# create the modified xml file by running
# python  add_time_corrections.py  /g/data/ha3/Passive/SHARED_DATA/GPS_Clock/corrections/AU.HTT_clock_correction.csv ../../tests/testdata/network_AU_0.xml 

def inspect_stations( input_station_xml ):
    
    our_inv = read_inventory(input_station_xml,format='STATIONXML')

    #csv_str = our_inv.networks[0].stations[0].extra.gpsclockcorrection.value

    stations =our_inv.networks[0].stations
    print (input_station_xml)
    
    if len(stations)>=2: 
        print ("Warning: more than one Station Node = %s" %len(stations), stations[0],stations[1])
    else:
        print ("Just one Station Node = %s" %len(stations))
    
    return stations


In [31]:
#input_station_xml ="/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/AU.HTT_station_inv_modified.xml"
#inspect_stations(input_station_xml)

In [40]:

import os, glob

xmlfolder = "/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/"
xmlfolder = "/Datasets/Station_Extra_Metadata/NewInventoryXML" 
# dirpath # "/g/data/ha3/Passive/SHARED_DATA/Inventory/Station_Extra_Metadata/NewInventoryXML"
xmlfolder ="/Datasets/InventoryXml/OA_stations_2017-2018"

for axmlfile in glob.glob(os.path.join(xmlfolder,"*.xml")):
    
    stations = inspect_stations(axmlfile)
    
#     if len(stations)>=2:
#         for astation in stations:
#             print("### ", astation.code, astation.extra)

/Datasets/InventoryXml/OA_stations_2017-2018/OA_CE23_station2.xml
	Station Code: CE23
	Channel Count: None/None (Selected/Total)
	2017-11-04T03:42:16.000000Z - 2018-06-06T00:26:04.000000Z
	Access: None 
	Latitude: -19.00, Longitude: 139.00, Elevation: 158.6 m
	Available Channels:
		CE23.0M.HHZ, CE23.0M.HHN, CE23.0M.HHE Station CE23 (CE23)
	Station Code: CE23
	Channel Count: None/None (Selected/Total)
	2018-06-06T00:41:08.000000Z - 2018-11-18T21:12:33.000000Z
	Access: None 
	Latitude: -19.00, Longitude: 139.00, Elevation: 158.6 m
	Available Channels:
		CE23.0M.HHZ, CE23.0M.HHN, CE23.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_CD21_station.xml
	Station Code: CD21
	Channel Count: None/None (Selected/Total)
	2017-10-06T00:39:17.000000Z - 2018-06-04T20:25:33.000000Z
	Access: None 
	Latitude: -18.01, Longitude: 138.49, Elevation: 78.0 m
	Available Channels:
		CD21.0M.HHZ, CD21.0M.HHN, CD21.0M.HHE Station CD21 (CD21)
	Station Code: CD21
	Channel Count: None/None (Selected/Total)
	2

/Datasets/InventoryXml/OA_stations_2017-2018/OA_CA22_station2.xml
	Station Code: CA22
	Channel Count: None/None (Selected/Total)
	2017-09-21T00:43:14.000000Z - 2018-06-13T03:31:58.000000Z
	Access: None 
	Latitude: -18.50, Longitude: 137.00, Elevation: 295.2 m
	Available Channels:
		CA22.0M.HHZ, CA22.0M.HHN, CA22.0M.HHE Station CA22 (CA22)
	Station Code: CA22
	Channel Count: None/None (Selected/Total)
	2018-06-13T03:36:26.000000Z - 2018-08-15T23:29:29.000000Z
	Access: None 
	Latitude: -18.50, Longitude: 137.00, Elevation: 295.2 m
	Available Channels:
		CA22.0M.HHZ, CA22.0M.HHN, CA22.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_CI23_station.xml
	Station Code: CI23
	Channel Count: None/None (Selected/Total)
	2017-11-05T05:41:48.000000Z - 2018-06-06T23:54:38.000000Z
	Access: None 
	Latitude: -19.02, Longitude: 141.00, Elevation: 43.8 m
	Available Channels:
		CI23.0M.HHZ, CI23.0M.HHN, CI23.0M.HHE Station CI23 (CI23)
	Station Code: CI23
	Channel Count: None/None (Selected/Total)
	2

/Datasets/InventoryXml/OA_stations_2017-2018/OA_BW28_station.xml
	Station Code: BW28
	Channel Count: None/None (Selected/Total)
	2017-09-24T23:46:39.000000Z - 2018-07-02T23:43:24.000000Z
	Access: None 
	Latitude: -21.51, Longitude: 135.01, Elevation: 400.1 m
	Available Channels:
		BW28.0M.HHZ, BW28.0M.HHN, BW28.0M.HHE Station BW28 (BW28)
	Station Code: BW28
	Channel Count: None/None (Selected/Total)
	2018-07-02T23:48:55.000000Z - 2018-08-16T23:39:41.000000Z
	Access: None 
	Latitude: -21.51, Longitude: 135.01, Elevation: 400.1 m
	Available Channels:
		BW28.0M.HHZ, BW28.0M.HHN, BW28.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_BS28_station2.xml
	Station Code: BS28
	Channel Count: None/None (Selected/Total)
	2017-09-26T02:02:42.000000Z - 2018-07-03T02:37:29.000000Z
	Access: None 
	Latitude: -21.51, Longitude: 132.99, Elevation: 511.1 m
	Available Channels:
		BS28.0M.HHZ, BS28.0M.HHN, BS28.0M.HHE Station BS28 (BS28)
	Station Code: BS28
	Channel Count: None/None (Selected/Total)
	

/Datasets/InventoryXml/OA_stations_2017-2018/OA_CA28_station2.xml
	Station Code: CA28
	Channel Count: None/None (Selected/Total)
	2017-09-17T00:54:07.000000Z - 2018-06-15T06:31:43.000000Z
	Access: None 
	Latitude: -21.50, Longitude: 136.99, Elevation: 249.0 m
	Available Channels:
		CA28.0M.HHZ, CA28.0M.HHN, CA28.0M.HHE Station CA28 (CA28)
	Station Code: CA28
	Channel Count: None/None (Selected/Total)
	2018-06-15T06:36:35.000000Z - 2018-08-15T03:00:19.000000Z
	Access: None 
	Latitude: -21.50, Longitude: 136.99, Elevation: 249.0 m
	Available Channels:
		CA28.0M.HHZ, CA28.0M.HHN, CA28.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_BZ22_station2.xml
	Station Code: BZ22
	Channel Count: None/None (Selected/Total)
	2017-09-21T01:54:23.000000Z - 2018-03-18T18:22:11.000000Z
	Access: None 
	Latitude: -18.49, Longitude: 136.49, Elevation: 236.4 m
	Available Channels:
		BZ22.0M.HHZ, BZ22.0M.HHN, BZ22.0M.HHE Station BZ22 (BZ22)
	Station Code: BZ22
	Channel Count: None/None (Selected/Total)


/Datasets/InventoryXml/OA_stations_2017-2018/OA_CH23_station2.xml
	Station Code: CH23
	Channel Count: None/None (Selected/Total)
	2017-11-05T06:20:59.000000Z - 2018-06-06T23:12:55.000000Z
	Access: None 
	Latitude: -19.00, Longitude: 140.50, Elevation: 51.5 m
	Available Channels:
		CH23.0M.HHZ, CH23.0M.HHN, CH23.0M.HHE Station CH23 (CH23)
	Station Code: CH23
	Channel Count: None/None (Selected/Total)
	2018-06-06T23:18:43.000000Z - 2018-08-13T01:29:45.000000Z
	Access: None 
	Latitude: -19.00, Longitude: 140.50, Elevation: 51.5 m
	Available Channels:
		CH23.0M.HHZ, CH23.0M.HHN, CH23.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_BX20_station2.xml
Just one Station Node = 1
/Datasets/InventoryXml/OA_stations_2017-2018/OA_BX24_station.xml
	Station Code: BX24
	Channel Count: None/None (Selected/Total)
	2017-09-20T06:14:16.000000Z - 2018-06-15T21:45:15.000000Z
	Access: None 
	Latitude: -19.50, Longitude: 135.50, Elevation: 218.9 m
	Available Channels:
		BX24.0M.HHZ, BX24.0M.HHN, BX24.0

/Datasets/InventoryXml/OA_stations_2017-2018/OA_CA22_station.xml
	Station Code: CA22
	Channel Count: None/None (Selected/Total)
	2017-09-21T00:43:14.000000Z - 2018-06-13T03:31:58.000000Z
	Access: None 
	Latitude: -18.50, Longitude: 137.00, Elevation: 295.2 m
	Available Channels:
		CA22.0M.HHZ, CA22.0M.HHN, CA22.0M.HHE Station CA22 (CA22)
	Station Code: CA22
	Channel Count: None/None (Selected/Total)
	2018-06-13T03:36:26.000000Z - 2018-08-15T23:29:29.000000Z
	Access: None 
	Latitude: -18.50, Longitude: 137.00, Elevation: 295.2 m
	Available Channels:
		CA22.0M.HHZ, CA22.0M.HHN, CA22.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_CC22_station2.xml
	Station Code: CC22
	Channel Count: None/None (Selected/Total)
	2017-09-18T00:26:52.000000Z - 2018-04-28T20:00:59.000000Z
	Access: None 
	Latitude: -18.64, Longitude: 137.94, Elevation: 258.4 m
	Available Channels:
		CC22.0M.HHZ, CC22.0M.HHN, CC22.0M.HHE Station CC22 (CC22)
	Station Code: CC22
	Channel Count: None/None (Selected/Total)
	

/Datasets/InventoryXml/OA_stations_2017-2018/OA_BU26_station.xml
	Station Code: BU26
	Channel Count: None/None (Selected/Total)
	2017-09-26T22:31:06.000000Z - 2018-07-02T04:13:15.000000Z
	Access: None 
	Latitude: -20.56, Longitude: 134.02, Elevation: 333.1 m
	Available Channels:
		BU26.0M.HHZ, BU26.0M.HHN, BU26.0M.HHE Station BU26 (BU26)
	Station Code: BU26
	Channel Count: None/None (Selected/Total)
	2018-07-02T13:01:40.000000Z - 2018-08-18T03:33:17.000000Z
	Access: None 
	Latitude: -20.56, Longitude: 134.02, Elevation: 333.1 m
	Available Channels:
		BU26.0M.HHZ, BU26.0M.HHN, BU26.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_CC24_station2.xml
	Station Code: CC24
	Channel Count: None/None (Selected/Total)
	2017-09-17T22:03:33.000000Z - 2018-06-12T04:46:14.000000Z
	Access: None 
	Latitude: -19.50, Longitude: 137.99, Elevation: 263.7 m
	Available Channels:
		CC24.0M.HHZ, CC24.0M.HHN, CC24.0M.HHE Station CC24 (CC24)
	Station Code: CC24
	Channel Count: None/None (Selected/Total)
	

		BT23.0M.HHZ, BT23.0M.HHN, BT23.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_BX27_station.xml
	Station Code: BX27
	Channel Count: None/None (Selected/Total)
	2017-09-25T02:37:38.000000Z - 2018-07-02T00:50:35.000000Z
	Access: None 
	Latitude: -21.00, Longitude: 135.50, Elevation: 371.1 m
	Available Channels:
		BX27.0M.HHZ, BX27.0M.HHN, BX27.0M.HHE Station BX27 (BX27)
	Station Code: BX27
	Channel Count: None/None (Selected/Total)
	2018-07-02T00:56:49.000000Z - 2018-08-17T00:24:05.000000Z
	Access: None 
	Latitude: -21.00, Longitude: 135.50, Elevation: 371.1 m
	Available Channels:
		BX27.0M.HHZ, BX27.0M.HHN, BX27.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_BT23_station2.xml
	Station Code: BT23
	Channel Count: None/None (Selected/Total)
	2017-09-28T04:11:41.000000Z - 2018-07-04T03:40:38.000000Z
	Access: None 
	Latitude: -18.99, Longitude: 133.49, Elevation: 284.3 m
	Available Channels:
		BT23.0M.HHZ, BT23.0M.HHN, BT23.0M.HHE Station BT23 (BT23)
	Station Code: BT23
	Cha

/Datasets/InventoryXml/OA_stations_2017-2018/OA_BY21_station.xml
	Station Code: BY21
	Channel Count: None/None (Selected/Total)
	2017-10-01T00:21:41.000000Z - 2018-02-27T10:30:32.000000Z
	Access: None 
	Latitude: -17.99, Longitude: 136.00, Elevation: 229.0 m
	Available Channels:
		BY21.0M.HHZ, BY21.0M.HHN, BY21.0M.HHE Station BY21 (BY21)
	Station Code: BY21
	Channel Count: None/None (Selected/Total)
	2018-06-15T01:12:07.000000Z - 2018-08-16T03:28:52.000000Z
	Access: None 
	Latitude: -17.99, Longitude: 136.00, Elevation: 229.0 m
	Available Channels:
		BY21.0M.HHZ, BY21.0M.HHN, BY21.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_CB28_station.xml
	Station Code: CB28
	Channel Count: None/None (Selected/Total)
	2017-09-16T23:39:02.000000Z - 2018-06-11T01:38:31.000000Z
	Access: None 
	Latitude: -21.50, Longitude: 137.51, Elevation: 220.7 m
	Available Channels:
		CB28.0M.HHZ, CB28.0M.HHN, CB28.0M.HHE Station CB28 (CB28)
	Station Code: CB28
	Channel Count: None/None (Selected/Total)
	2

/Datasets/InventoryXml/OA_stations_2017-2018/OA_CD28_station2.xml
	Station Code: CD28
	Channel Count: None/None (Selected/Total)
	2017-11-06T03:02:15.000000Z - 2018-06-11T02:39:00.000000Z
	Access: None 
	Latitude: -21.51, Longitude: 138.49, Elevation: 197.3 m
	Available Channels:
		CD28.0M.HHZ, CD28.0M.HHN, CD28.0M.HHE Station CD28 (CD28)
	Station Code: CD28
	Channel Count: None/None (Selected/Total)
	2018-06-11T02:44:16.000000Z - 2018-08-12T04:55:16.000000Z
	Access: None 
	Latitude: -21.51, Longitude: 138.49, Elevation: 197.3 m
	Available Channels:
		CD28.0M.HHZ, CD28.0M.HHN, CD28.0M.HHE
/Datasets/InventoryXml/OA_stations_2017-2018/OA_BX24_station2.xml
	Station Code: BX24
	Channel Count: None/None (Selected/Total)
	2017-09-20T06:14:16.000000Z - 2018-06-15T21:45:15.000000Z
	Access: None 
	Latitude: -19.50, Longitude: 135.50, Elevation: 218.9 m
	Available Channels:
		BX24.0M.HHZ, BX24.0M.HHN, BX24.0M.HHE Station BX24 (BX24)
	Station Code: BX24
	Channel Count: None/None (Selected/Total)


# Check the output statoinXML file from ASDF 

- to ensure they are identical to the ingested station xml
- in particular, the extra station xml file can be recovered


In [33]:
!skip this section

/bin/sh: 1: skip: not found


In [34]:
# Check the output statoinXML file from ASDF

# We have added the input station xml into an ASDF file, then extract it out. See pyasdf_tests.ipynb
# https://github.com/SeismicData/pyasdf/issues/63

    
from obspy import read_inventory

input_station_xml ="OA.CF28_station_inv_modified_json_extracted_from_ASDF.xml"
our_inv = read_inventory(input_station_xml,format='STATIONXML')
# print(our_inv.networks[0].stations[0].extra)

stn_meta0 = our_inv.networks[0].stations[0]


print(type(stn_meta0.extra))
print(stn_meta0.extra)

print(stn_meta0.start_date)
print(stn_meta0.end_date)

print(stn_meta0.equipments)

FileNotFoundError: [Errno 2] No such file or directory: 'OA.CF28_station_inv_modified_json_extracted_from_ASDF.xml'